# Importing SimaPro CSV files

This notebooks demonstrates how to use the `bw2io.SimaProBlocCSVImporter` to import LCI created in SimaPro into brightway. 
It uses brightway 2.5.

The notebook is split in 2 sections:

1. Importing an LCI that only has references (i.e. _Exchanges_) to either Biosphere flows or to itself.
2. Importing an LCI that includes references to another "`library`" (as SimaPro calls them) like ecoinvent.

In [33]:
from pathlib import Path

import bw2data as bd
import bw2io as bi
import bw_simapro_csv # only import it here to printe the version of the module. We will use it through bw2io
from pprint import pprint

In [2]:
print(f"Using bw2data version: {bd.__version__}")
print(f"Using bw2io version: {bi.__version__}")
print(f"Using bw_simapro_csv version: {bw_simapro_csv.__version__}")

Using bw2data version: (4, 4)
Using bw2io version: 0.9.5
Using bw_simapro_csv version: 0.4.2


## The imported inventories
The LCIs created in simapro mimic the LCI of a bike as depicted in [from the ground up notebooks](https://github.com/brightway-lca/from-the-ground-up/).

### Bike
The first inventory exported from SimaPro is the [bike_example_no_ei.CSV](bike_example_no_ei.CSV) file. It only includes the following products:

+ Bike
+ Carbon fibre
+ Natural gas

and their corresponding "Processes":

+ Bike production
+ CF production
+ NG Production

Below is a graphical depiction of the supply chain:

![supply-chain-simple.png](supply-chain-simple.png)

### Bike 2

This inventory, additionally includes as input electricity from Norway as an input (i.e. "_exchange_") for the `CF production` process.



## Importing LCIs with only self references

In [3]:
bd.projects

Brightway2 projects manager with 2 objects:
	default
	ecoinvent311
Use `projects.report()` to get a report on all projects.

In [4]:
bd.projects.set_current("ecoinvent311")
# This project already includes ecoinvent 3.11 cutoff, which is necessary for section 2
bd.databases

Databases dictionary with 4 object(s):
	bike_db
	bike_example
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-cutoff

### Use the _real_ SimaPro importer that does all the magic

Although the `bw_simapro_csv` [package](https://github.com/brightway-lca/bw_simapro_csv) has the main functionalities that parse and interpret the SimaPro CSV, it is not a "regular" brightway importer. That is, it does not in clude `strategies` to be applied (for example, to normalize units, or to parse the names of ecoinvent extracting the location from them.

The right way to proceed is to use the `bi.SimaProBlockCSVImporter`, apply the strategies and then fiddle the data if there are unlinked exchanges.

In [5]:
spi = bi.SimaProBlockCSVImporter(Path("bike_example_no_ei.CSV"))

2025-08-08 07:08:52.390 | INFO     | bw_simapro_csv.main:__init__:119 - Writing logs to /root/.local/state/bw_simapro_csv/log/bike_example_no_ei-2025-08-08T07-08-52
2025-08-08 07:08:52.404 | INFO     | bw_simapro_csv.main:__init__:142 - Using database name 'bike_example'
2025-08-08 07:08:52.405 | INFO     | bw_simapro_csv.main:__init__:147 - SimaPro CSV import started.
	File: 'bike_example_no_ei.CSV'
	Delimiter: ';'
	Name: 'bike_example'
2025-08-08 07:08:52.408 | INFO     | bw_simapro_csv.main:resolve_parameters:324 - Extracted and cleaned 3 process datasets


In [6]:
# Do some well known changes to data, based on simapro standards by applying the strategies.
spi.apply_strategies()

Applying strategy: set_metadata_using_single_functional_exchange
Applying strategy: override_process_name_using_single_functional_exchange
Applying strategy: drop_unspecified_subcategories
Applying strategy: split_simapro_name_geo
Applying strategy: link_iterable_by_fields
Applying strategy: match_internal_simapro_simapro_with_unit_conversion
Applied 6 strategies in 0.62 seconds


In [7]:
# Verify the status of the importer
# We must verify here that we have:
# 6 graph nodes (3 for the products, 3 for the processes) and 
# 6 edges (see the supply chain scheme above)
spi.statistics()

Graph statistics for `bike_example` importer:
6 graph nodes:
	process: 3
	product: 3
6 graph edges:
	production: 3
	technosphere: 2
	biosphere: 1
3 edges to the following databases:
	bike_example: 3
3 unique unlinked edges (3 total):
	technosphere: 2
	biosphere: 1




(6, 6, 3, 0)

At this point, the `spi` importer shows that we have unlinked edges, because we haven't matched yet the data neither to the own database we are trying to import, nor to the `ecoinvent-3.11-biosphere` (nor `ecoinvent-3.11-cutoff`, but this is for section 3)

### Matching

#### Match the database we are importing against itself

In [8]:
spi.match_database()

Applying strategy: link_iterable_by_fields


In [9]:
# We verify again the status of the importer
spi.statistics()

Graph statistics for `bike_example` importer:
6 graph nodes:
	process: 3
	product: 3
6 graph edges:
	production: 3
	technosphere: 2
	biosphere: 1
5 edges to the following databases:
	bike_example: 5
1 unique unlinked edges (1 total):
	biosphere: 1




(6, 6, 1, 0)

At this point, normally, all self references to the database we are importing should be resolved, and we should only have one unlinked edge related to the biosphere.

#### Match against the biosphere

In [10]:
spi.match_database("ecoinvent-3.11-biosphere")

Applying strategy: link_iterable_by_fields


In [11]:
spi.statistics()

Graph statistics for `bike_example` importer:
6 graph nodes:
	process: 3
	product: 3
6 graph edges:
	production: 3
	technosphere: 2
	biosphere: 1
5 edges to the following databases:
	bike_example: 5
1 unique unlinked edges (1 total):
	biosphere: 1




(6, 6, 1, 0)

At this point, we know there is one biosphere flow that was not automagically matched before.
We can print to see which flow it is

In [12]:
# print the unlinked flows
# We know it's CO2
for u in spi.unlinked:
    print(u)

{'name': 'Carbon dioxide', 'context': ('Emissions to air', ''), 'unit': 'kg', 'line_no': 229, 'amount': 26.6, 'uncertainty type': 0, 'loc': 26.6, 'cas_number': '124-38-9', 'comment': 'Formula: CO2', 'type': 'biosphere'}


In [13]:
# find CO2 from the biosphere
for flow in bd.Database("ecoinvent-3.11-biosphere"):
    if "Carbon dioxide, fossil" in flow["name"] and flow["categories"] == ("air",):
        print(flow.as_dict())
        co2_flow = flow

{'categories': ('air',), 'code': '349b29d1-3e58-4c66-98b9-9d1a076efd2e', 'CAS number': '000124-38-9', 'synonyms': ['Carbon dioxide'], 'name': 'Carbon dioxide, fossil', 'database': 'ecoinvent-3.11-biosphere', 'unit': 'kilogram', 'type': 'emission', 'id': 139717284728938584}


In [14]:
# add as input the CO2 to the exchanges in the imported data
for p in spi.data:
    for e in p.get("exchanges", []):
        if e["type"] == "biosphere":
            e["input"] = co2_flow.key

In [15]:
spi.statistics()

Graph statistics for `bike_example` importer:
6 graph nodes:
	process: 3
	product: 3
6 graph edges:
	production: 3
	technosphere: 2
	biosphere: 1
6 edges to the following databases:
	bike_example: 5
	ecoinvent-3.11-biosphere: 1
0 unique unlinked edges (0 total):




(6, 6, 0, 0)

Now that the importer reports no unlinked edges, we can proceed to write the database and do some first tests.

In [16]:
spi.write_database()

07:11:53 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 7928.74it/s]

07:11:53 [info     ] Vacuuming database            


Created database: bike_example


Brightway2 SQLiteBackend: bike_example

In [18]:
for p in bd.Database("bike_example"):
    if p["type"] == "product" and p['name'] == 'Bike':
        print(p)        
        bike_p = p

'Bike' (p, None, None)


In [19]:
import bw2calc as bc

In [20]:
ef_method_cc = [m for m in bd.methods if m[1] == "EF v3.1" and m[2] == "climate change"]
ef_method_cc

[('ecoinvent-3.11',
  'EF v3.1',
  'climate change',
  'global warming potential (GWP100)')]

In [21]:
functional_unit, data_objs, _ = bd.prepare_lca_inputs(
    {bike_p: 1}, method=ef_method_cc[0],remapping=False
)

In [22]:
lca = bc.LCA(demand=functional_unit, data_objs=data_objs)
lca.lci()
lca.lcia()
lca.score

66.50000095367432

## Importing LCIs with external references

In [79]:
ext_spi = bi.SimaProBlockCSVImporter(Path('bike_example_with_ei.CSV'))

2025-08-08 07:33:09.039 | INFO     | bw_simapro_csv.main:__init__:119 - Writing logs to /root/.local/state/bw_simapro_csv/log/bike_example_with_ei-2025-08-08T07-33-09
2025-08-08 07:33:09.052 | INFO     | bw_simapro_csv.main:__init__:142 - Using database name 'bike_example'
2025-08-08 07:33:09.052 | INFO     | bw_simapro_csv.main:__init__:147 - SimaPro CSV import started.
	File: 'bike_example_with_ei.CSV'
	Delimiter: ';'
	Name: 'bike_example'
2025-08-08 07:33:09.055 | INFO     | bw_simapro_csv.main:resolve_parameters:324 - Extracted and cleaned 4 process datasets


In [80]:
#ext_spi.use_ecoinvent_strategies()
ext_spi.apply_strategies()

Applying strategy: set_metadata_using_single_functional_exchange
Applying strategy: override_process_name_using_single_functional_exchange
Applying strategy: drop_unspecified_subcategories
Applying strategy: split_simapro_name_geo
Applying strategy: link_iterable_by_fields
Applying strategy: match_internal_simapro_simapro_with_unit_conversion
Applied 6 strategies in 0.53 seconds


In [81]:
ext_spi.statistics()

Graph statistics for `bike_example` importer:
8 graph nodes:
	process: 4
	product: 4
9 graph edges:
	technosphere: 4
	production: 4
	biosphere: 1
4 edges to the following databases:
	bike_example: 4
4 unique unlinked edges (5 total):
	technosphere: 3
	biosphere: 1




(8, 9, 5, 0)

### Match the databases

In [82]:
# First, against itself
ext_spi.match_database()

Applying strategy: link_iterable_by_fields


In [83]:
ext_spi.statistics()

Graph statistics for `bike_example` importer:
8 graph nodes:
	process: 4
	product: 4
9 graph edges:
	technosphere: 4
	production: 4
	biosphere: 1
7 edges to the following databases:
	bike_example: 7
2 unique unlinked edges (2 total):
	technosphere: 1
	biosphere: 1




(8, 9, 2, 0)

In [84]:
# add as input the CO2 to the exchanges in the imported data
for p in ext_spi.data:
    for e in p.get("exchanges", []):
        if e["type"] == "biosphere":
            e["input"] = co2_flow.key
            e["categories"] = co2_flow['categories']

In [85]:
ext_spi.match_database('ecoinvent-3.11-cutoff', fields=["name", "unit", "location", "reference product"])
ext_spi.statistics()

Applying strategy: link_iterable_by_fields
Graph statistics for `bike_example` importer:
8 graph nodes:
	process: 4
	product: 4
9 graph edges:
	technosphere: 4
	production: 4
	biosphere: 1
8 edges to the following databases:
	bike_example: 7
	ecoinvent-3.11-biosphere: 1
1 unique unlinked edges (1 total):
	technosphere: 1




(8, 9, 1, 0)

In [86]:
ext_spi.use_ecoinvent_strategies()
ext_spi.apply_strategies()

Applying strategy: set_metadata_using_single_functional_exchange
Applying strategy: drop_unspecified_subcategories
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: split_simapro_name_geo
Applying strategy: strip_biosphere_exc_locations
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: set_code_by_activity_hash
Applying strategy: change_electricity_unit_mj_to_kwh
Applying strategy: link_technosphere_based_on_name_unit_location
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: normalize_simapro_biosphere_names
Applying strategy: migrate_exchanges
Applying strategy: fix_localized_water_flows
Applying strategy: link_iterable_by_fields
Applied 18 strategies in 0.21 seconds


In [87]:
ext_spi.statistics()

Graph statistics for `bike_example` importer:
8 graph nodes:
	process: 4
	product: 4
9 graph edges:
	technosphere: 4
	production: 4
	biosphere: 1
8 edges to the following databases:
	bike_example: 7
	ecoinvent-3.11-biosphere: 1
1 unique unlinked edges (1 total):
	technosphere: 1




(8, 9, 1, 0)

The biosphere flow seems to be well replaced, but now the only technosphere flow is missing (presumable the one from ecoinvent).
Let's look at it in detail.

In [88]:
for u in ext_spi.unlinked:
    pprint(u)

{'amount': 40.0,
 'comment': '',
 'line_no': 225,
 'loc': 40.0,
 'name': 'Electricity, medium voltage {NO}| market for electricity, medium '
         'voltage | Cut-off, U',
 'original unit before conversion': 'kWh',
 'simapro_category': 'Materials/fuels',
 'type': 'technosphere',
 'uncertainty type': 0,
 'unit': 'kilowatt hour',
 'unit conversion factor': 3.6}


In [89]:
ext_spi.match_database('ecoinvent-3.11-cutoff')

Applying strategy: link_iterable_by_fields


In [90]:
ext_spi.statistics()

Graph statistics for `bike_example` importer:
8 graph nodes:
	process: 4
	product: 4
9 graph edges:
	technosphere: 4
	production: 4
	biosphere: 1
8 edges to the following databases:
	bike_example: 7
	ecoinvent-3.11-biosphere: 1
1 unique unlinked edges (1 total):
	technosphere: 1




(8, 9, 1, 0)

In [92]:
resulting_excel = ext_spi.write_excel()

Wrote matching file to:
/root/.local/share/Brightway3/ecoinvent311.bf5aadba/output/db-matching-bike_example.xlsx


In [93]:
import shutil
import os

In [96]:
# Destination is the current working directory
destination = Path.cwd() / resulting_excel.name

# Copy the file
shutil.copy(resulting_excel, destination)

print(f"Copied {resulting_excel} to {destination}")

Copied /root/.local/share/Brightway3/ecoinvent311.bf5aadba/output/db-matching-bike_example.xlsx to /opt/app/db-matching-bike_example.xlsx


In [97]:
# Find the electricity dataset we want
ei_db = bd.Database('ecoinvent-3.11-cutoff')

In [108]:
results = ei_db.search('market for electricity, medium voltage', limit=10000)
for r in results:
    if r['location'] == 'NO' and 'market for electricity, medium voltage' == r['name']:
        print(r)

'market for electricity, medium voltage' (kilowatt hour, NO, None)


In [118]:
for d in ext_spi.data:
    if 'exchanges' in d:
        for exc in d['exchanges']:            
            if exc['name'] == 'Electricity, medium voltage {NO}| market for electricity, medium voltage | Cut-off, U':
                print(exc)
                # add the input as it should
                exc['name'] = r['name']
                exc['input'] = r.key
        

In [119]:
spi.statistics()

Graph statistics for `bike_example` importer:
6 graph nodes:
	process: 3
	product: 3
6 graph edges:
	production: 3
	technosphere: 2
	biosphere: 1
6 edges to the following databases:
	bike_example: 5
	ecoinvent-3.11-biosphere: 1
0 unique unlinked edges (0 total):




(6, 6, 0, 0)

In [120]:
spi.write_database()

07:47:54 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 6400.26it/s]

07:47:54 [info     ] Vacuuming database            


Created database: bike_example


Brightway2 SQLiteBackend: bike_example